In [105]:
import pandas as pd
import numpy as np
import pprint
from nltk.tokenize import TweetTokenizer
#not needed right now (maybe ever?) doing one-hot encoding next cell
#might have to use if we are doing the neutral thing
s = """
#encodes a column of parties into -1, 0, or 1
def encode_party(party_series):
    types = {"Democrat": 0, "Neutral": 0, "Republican": 0}
    output_array = []
    for party in party_series:
        if party == "Democrat":
            types["Democrat"] += 1
        elif party == "Neutral":
            types["Neutral"] += 1
        elif party == "Republican":
            types["Republican"] += 1
            
    resulting_encoding = []

    for type_ in types.keys():
        resulting_encoding.append(type_)
    resulting_encoding.sort()
    return np.asarray(resulting_encoding)
"""

In [104]:
#Hydrating data
tweets = pd.read_csv("data/ExtractedTweets.csv")

#Preprocessing data

#one-hot encoding republican and democrat
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(dtype=np.int, sparse=True, categories='auto')
parties = pd.DataFrame(onehot.fit_transform(tweets[["Party"]])\
                            .toarray(),
                            columns=["Democrat", "Republican"]) #todo add neutral column & training data
parties["Handle"] = tweets.Handle
tweets = tweets.drop(["Party"], axis=1)
encoded_data = pd.merge(parties, tweets)


#Concatinates all tweets in a user's recent tweets, and tokenizes the result
def concatinate_tweets(all_tweets):
    document = " "
    for tweet in all_tweets:
        for single_tweet in tweet.values[0:200]: 
            document += single_tweet + " "
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(document)

#dictionary containing the user's handle and a string representation of 200 of their tweets
handles_and_tweets = {}
for handle in encoded_data.groupby(["Handle"])["Tweet"]:
    handles_and_tweets[handle[0]] = concatinate_tweets(handle[1:])



In [106]:
#Organizing the data for training here
df = pd.DataFrame.from_dict(handles_and_tweets, orient='index')
df = df.reset_index()
df.rename(columns={'index':'Handle'}, inplace=True)
finished_product = pd.merge(parties, df, how='right')
finished_product = finished_product.set_index('Handle')
finished_product = finished_product.drop_duplicates()
X = finished_product.iloc[0:, 2:]
Y = finished_product['Democrat']

In [107]:
# W2V encoding
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)


In [115]:
#tf-idf and google word2vec
#TODO: vectorize the tweets (for each value in handles_and_tweets.items())
# TODO : tf-idf

#word2vec vectorization
user_and_vectorized_tweets = {}
k = 0
for index, tweets in X.iterrows(): #todo maybe not all tweets are same length - might just be getting max and assigning that to all of them
    total_embeddings = np.zeros(300) # this number can be tweaked? maybe im not sure
    vectorized_tweet = []
    k+=1
    for word in tweets:
        try:
            encoding_of_word = word_vectors.get_vector(word)
        except KeyError:
            encoding_of_word = np.zeros(300)
        for i, embedding in enumerate(encoding_of_word):
            total_embeddings[i] = total_embeddings[i] + embedding
    for i in range(len(total_embeddings)):
        total_embeddings[i] = total_embeddings[i] / len(tweets)
        
    user_and_vectorized_tweets[index] = total_embeddings
    print(f'finished row {k}/433')
    
handle_and_vector = pd.DataFrame.from_dict(user_and_vectorized_tweets, orient='index')

finished row 1/433
finished row 2/433
finished row 3/433
finished row 4/433
finished row 5/433
finished row 6/433
finished row 7/433
finished row 8/433
finished row 9/433
finished row 10/433
finished row 11/433
finished row 12/433
finished row 13/433
finished row 14/433
finished row 15/433
finished row 16/433
finished row 17/433
finished row 18/433
finished row 19/433
finished row 20/433
finished row 21/433
finished row 22/433
finished row 23/433
finished row 24/433
finished row 25/433
finished row 26/433
finished row 27/433
finished row 28/433
finished row 29/433
finished row 30/433
finished row 31/433
finished row 32/433
finished row 33/433
finished row 34/433
finished row 35/433
finished row 36/433
finished row 37/433
finished row 38/433
finished row 39/433
finished row 40/433
finished row 41/433
finished row 42/433
finished row 43/433
finished row 44/433
finished row 45/433
finished row 46/433
finished row 47/433
finished row 48/433
finished row 49/433
finished row 50/433
finished 

finished row 413/433
finished row 414/433
finished row 415/433
finished row 416/433
finished row 417/433
finished row 418/433
finished row 419/433
finished row 420/433
finished row 421/433
finished row 422/433
finished row 423/433
finished row 424/433
finished row 425/433
finished row 426/433
finished row 427/433
finished row 428/433
finished row 429/433
finished row 430/433
finished row 431/433
finished row 432/433
finished row 433/433


In [116]:
#Splitting up the data into train and test
from sklearn.model_selection import train_test_split
                            #Insert formatted 2d array here vvv
X_train, X_test, Y_train, Y_test = train_test_split(handle_and_vector, Y, test_size = 0.2)


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
RepSires,0.000391,0.027553,0.023491,0.043056,-0.030019,-0.027537,0.008416,-0.036426,0.043970,0.025529,...,-0.039116,-0.010980,-0.051604,0.017615,-0.015469,-0.012483,-0.011499,-0.012291,0.030535,-0.008691
RepGaramendi,0.007959,0.031838,0.020584,0.052335,-0.030049,-0.020122,0.009018,-0.039879,0.047397,0.036155,...,-0.032955,0.002946,-0.050261,0.005462,-0.017188,-0.023676,-0.010230,-0.012759,0.020379,-0.013243
DrNealDunnFL2,0.004225,0.029977,0.015742,0.053479,-0.029000,-0.025518,0.007789,-0.032817,0.036691,0.036749,...,-0.044841,-0.012892,-0.061029,0.014197,-0.016566,-0.020461,-0.013814,-0.012997,0.035869,-0.016178
RepRussell,0.003855,0.028215,0.015704,0.039771,-0.014257,-0.025117,0.008009,-0.039296,0.037126,0.036870,...,-0.029881,-0.018334,-0.048769,0.017457,-0.016873,-0.033174,-0.019013,-0.015387,0.029805,-0.019067
RepThomasMassie,0.013473,0.020952,0.018559,0.050157,-0.034846,-0.015484,0.008920,-0.028619,0.036283,0.027747,...,-0.030226,0.002770,-0.041846,0.012793,-0.017261,-0.022832,-0.005424,-0.022065,0.017855,-0.000912
RepBobbyRush,0.004985,0.028899,0.020173,0.045154,-0.029215,-0.017072,0.012843,-0.029920,0.043020,0.027651,...,-0.040429,0.000431,-0.053394,0.011971,-0.019158,-0.016079,-0.003147,-0.016276,0.024828,-0.010225
RepJoseSerrano,-0.004713,0.025206,0.020230,0.046802,-0.027782,-0.017828,0.004035,-0.034814,0.035586,0.025126,...,-0.032007,-0.004085,-0.049811,0.019656,-0.014405,-0.021069,-0.016161,-0.014145,0.019848,-0.000931
DonaldNorcross,-0.004307,0.028218,0.018699,0.055407,-0.030486,-0.016289,0.005350,-0.040159,0.029833,0.028153,...,-0.037999,-0.000310,-0.057197,0.016883,-0.018459,-0.026211,-0.010889,-0.014987,0.019594,-0.005279
LloydDoggettTX,0.013852,0.024678,0.015363,0.050449,-0.032978,-0.016159,0.006747,-0.030439,0.034007,0.033142,...,-0.033292,-0.003088,-0.053658,0.008876,-0.009409,-0.024647,-0.004264,-0.019021,0.018568,-0.004124
RepHartzler,0.003865,0.026445,0.015298,0.050839,-0.021349,-0.023152,0.009403,-0.035624,0.036626,0.032269,...,-0.037525,-0.002097,-0.059814,0.012628,-0.013535,-0.025950,-0.013698,-0.009217,0.028896,-0.007145


In [117]:
#Training on model - MLP classifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
#Grid search CV
parameters = {'solver': ['lbfgs'],
              'max_iter': [1000,1500,2000 ],
              'alpha': 10.0 ** -np.arange(1, 10),
              'hidden_layer_sizes':np.arange(10, 15)}

tuned_MLP_classifier = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
tuned_MLP_classifier.fit(X_train, Y_train)
print(tuned_MLP_classifier.score(X_train.values, Y_train))
print(tuned_MLP_classifier.best_params_)

C:\Users\bklei\PythonWorkspace\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


1.0
{'alpha': 1e-06, 'hidden_layer_sizes': 12, 'max_iter': 1500, 'solver': 'lbfgs'}


C:\Users\bklei\PythonWorkspace\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [120]:
#Evaluation for MLP Classifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_predictions = tuned_MLP_classifier.predict(X_test)
print(confusion_matrix(Y_test.astype(int), y_predictions.astype(int)))
print(accuracy_score(Y_test.astype(int), y_predictions.astype(int)))


[[36  2]
 [ 6 43]]
0.9080459770114943


In [124]:
#Training on model - Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

#Grid search for hyperparameter tuning (for random forest)
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint.pprint(random_grid)
tuned_RF_classifier = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid,
                                  n_iter = 100, cv = 3, verbose=2, random_state=42,
                                  n_jobs = -1)# Fit the random search model
tuned_RF_classifier.fit(X_train, Y_train)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
C:\Users\bklei\PythonWorkspace\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   41.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.5min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [126]:
#Evaluation for RandomForest Classifier

#Evaluation:
print(f'Training Score: {tuned_RF_classifier.score(X_train, Y_train)} '
      + f'\n Validation Score: {tuned_RF_classifier.score(X_test, Y_test)}')

Training Score: 1.0 
 Validation Score: 0.8390804597701149
